In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import glob
import pandas as pd
import numpy as np
from sglm.helpers import filehelpers
from pathlib import Path

In [4]:
base_location = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\outputs')

# load_folder = base_location / Path(r'Figure_1_2')
# h5_beta_locations = str((load_folder / Path(r'all\f1*-ft\coefs\*_best_coeffs.h5')).resolve())
# load_folder = base_location / Path(r'Figure_3')
# h5_beta_locations = str((load_folder / Path(r'all\f3*-ft\coefs\*_best_coeffs.h5')).resolve())
# load_folder = base_location / Path(r'Figure_4_g1')
# h5_beta_locations = str((load_folder / Path(r'all\f4*-ft\coefs\*_best_coeffs.h5')).resolve())
# load_folder = base_location / Path(r'Figure_4_g2')
# h5_beta_locations = str((load_folder / Path(r'all\f4*-ft\coefs\*_best_coeffs.h5')).resolve())
load_folder = base_location / Path(r'Figure_5_g1')
h5_beta_locations = str((load_folder / Path(r'all\f5*-ft\coefs\*_best_coeffs.h5')).resolve())

out_folder = base_location / Path(r'final_outputs')

# h5_beta_locations = Path('/Users/josh/Dropbox (HMS)/jz/Tetanus/1194_1195_1214_1258_1259_2_best_coeffs.h5')
out_loc = str((out_folder).resolve())
h5_beta_locations = glob.glob(h5_beta_locations)

In [5]:
h5_lst = []
for h5_coef_fn in h5_beta_locations:
    h5f = pd.HDFStore(h5_coef_fn)
    h5fk = h5f.keys()
    for key in h5fk:
        h5_df = pd.read_hdf(h5f, key).reset_index()
        h5_df['fn'] = h5_coef_fn
        h5_df['key'] = key
        h5_df = h5_df.set_index(['fn','key'])
        
        
        h5_df = h5_df.sort_values('channel_name')
        h5_df['channel'] = h5_df['channel_name'].str.split('_').apply(lambda x: x[0])
        h5_df['run_num'] = h5_df['channel_name'].str.split('_').apply(lambda x: x[-1])
        
        h5_lst.append(h5_df)
concat = pd.concat(h5_lst, axis=0)

In [6]:
filehelpers.create_folder_if_not_exists(out_loc)

In [7]:
base_keys = list({
                'gACH':(0,0),
                'rDA':(0,0),
                'gDA':(0,0),
                'photometryCenterInIndex':(0,0),
                'photometryCenterOutIndex':(0,0),
                'photometrySideInIndex':(0,0),
                'photometrySideInIndexr':(0,0),
                'photometrySideOutIndex':(0,0),
                'sl': (0,0),
                'spnnrOff': (0,0),
                'photometryCenterInIndex':(0,0),
                'photometryCenterOutIndex':(0,0),
                'photometrySideInIndexAA':(0,0),
                'photometrySideInIndexAa':(0,0),
                'photometrySideInIndexaA':(0,0),
                'photometrySideInIndexaa':(0,0),
                'photometrySideInIndexAB':(0,0),
                'photometrySideInIndexAb':(0,0),
                'photometrySideInIndexaB':(0,0),
                'photometrySideInIndexab':(0,0),
                'photometrySideOutIndex':(0,0),
                'sl': (0,0),
                'spnnrOff': (0,0),
                }.keys())
# base_keys

In [8]:
def get_kernel_from_basename(base_name, column_names):
    kernel = [(_, int(_.split('_')[-1])) for _ in column_names if base_name+'_' == _[:len(base_name)+1]]
    kernel = sorted(kernel, key=lambda x: x[-1])
    kernel = [_[0] for _ in kernel]
    return kernel

all_kernels = []
for base_key in base_keys:
    all_kernels += get_kernel_from_basename(base_key, list(concat.columns))

In [9]:
gb_columns = ['mouse_id', 'channel', 'name']
kernels = (concat.reset_index().set_index(['mouse_id', 'channel', 'name', 'run_num']))[all_kernels].groupby(gb_columns).agg([np.mean, np.size, np.std])
kernels = kernels.swaplevel(axis=1)
# kernels['lb'] = kernels['mean'] - kernels['std']/np.sqrt(kernels['size'])
# kernels['mn'] = kernels['mean']
# kernels['ub'] = kernels['mean'] + kernels['std']/np.sqrt(kernels['size'])

In [10]:
(concat.reset_index().set_index(['mouse_id', 'channel', 'name', 'run_num']))[all_kernels].to_csv(str((Path(out_loc) / f'{load_folder.parts[-1]}-kernel_raw.csv').resolve()))

In [11]:
# entry

In [12]:
entries = []
for col in ['mean', 'size', 'std']:
    entry = kernels.unstack().unstack()[col]
    entry.index = [col]
    entries.append(entry)
stk = pd.concat(entries, axis=0).swaplevel(0,1,axis=1).swaplevel(0,2,axis=1)

In [14]:
# pd.read_csv(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\outputs\final_outputs\Figure_1_2-kernel_mn_size_std.csv', index_col=[0], header=[0,1,2])

In [15]:
stk.to_csv(str((Path(out_loc) / f'{load_folder.parts[-1]}-kernel_mn_size_std.csv').resolve()))

In [16]:
stk2 = stk.T.copy()
stk2['lb'] = stk2['mean'] - 1.96*stk2['std']/np.sqrt(stk2['size'])
stk2['mn'] = stk2['mean']
stk2['ub'] = stk2['mean'] + 1.96*stk2['std']/np.sqrt(stk2['size'])

In [17]:
stk2.to_csv(str((Path(out_loc) / f'{load_folder.parts[-1]}-kernel_mn_size_std_lb_mn_ub.csv').resolve()))

In [18]:
stk3 = stk2.swaplevel(-1, -2, axis=0).unstack()[['mn', 'lb', 'ub']].swaplevel(0,1,axis=1)[all_kernels].swaplevel(0,1,axis=1)[['mn', 'lb', 'ub']].copy()

In [19]:
stk3.to_csv(str((Path(out_loc) / f'{load_folder.parts[-1]}-kernel_mn_lb_ub.csv').resolve()))